# Primitives examples

<details>
<summary><b>Package versions</b></summary>

The code on this page was developed using the following requirements.
We recommend using these versions or newer.

```
qiskit[all]~=2.1.0
qiskit-ibm-runtime~=0.40.1
```
</details>

The examples in this section illustrate some common ways to use primitives. Before running these examples, follow the instructions in [Install and set up.](install-qiskit)

<Admonition type="note">
    These examples all use the primitives from Qiskit Runtime, but you could use the base primitives instead.
</Admonition>

## Estimator examples

Efficiently calculate and interpret expectation values of the quantum operators required for many algorithms with Estimator. Explore uses in molecular modeling, machine learning, and complex optimization problems.

### Run a single experiment

Use Estimator to determine the expectation value of a single circuit-observable pair.

In [1]:
import numpy as np
from qiskit.circuit.library import iqp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp, random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator

n_qubits = 50

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

mat = np.real(random_hermitian(n_qubits, seed=1234))
circuit = iqp(mat)
observable = SparsePauliOp("Z" * 50)

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)
isa_observable = observable.apply_layout(isa_circuit.layout)

estimator = Estimator(mode=backend)
job = estimator.run([(isa_circuit, isa_observable)])
result = job.result()

print(f" > Expectation value: {result[0].data.evs}")
print(f" > Metadata: {result[0].metadata}")

 > Expectation value: 0.0
 > Metadata: {'shots': 4096, 'target_precision': 0.015625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32}


### Run multiple experiments in a single job

Use Estimator to determine the expectation values of multiple circuit-observable pairs.

In [2]:
import numpy as np
from qiskit.circuit.library import iqp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp, random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator

n_qubits = 50

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

rng = np.random.default_rng()
mats = [np.real(random_hermitian(n_qubits, seed=rng)) for _ in range(3)]

pubs = []
circuits = [iqp(mat) for mat in mats]
observables = [
    SparsePauliOp("X" * 50),
    SparsePauliOp("Y" * 50),
    SparsePauliOp("Z" * 50),
]

# Get ISA circuits
pm = generate_preset_pass_manager(optimization_level=1, backend=backend)

for qc, obs in zip(circuits, observables):
    isa_circuit = pm.run(qc)
    isa_obs = obs.apply_layout(isa_circuit.layout)
    pubs.append((isa_circuit, isa_obs))

estimator = Estimator(backend)
job = estimator.run(pubs)
job_result = job.result()

for idx in range(len(pubs)):
    pub_result = job_result[idx]
    print(f">>> Expectation values for PUB {idx}: {pub_result.data.evs}")
    print(f">>> Standard errors for PUB {idx}: {pub_result.data.stds}")

>>> Expectation values for PUB 0: 0.004540295119182747
>>> Standard errors for PUB 0: 0.16071434944948873
>>> Expectation values for PUB 1: -0.09400123685837972
>>> Standard errors for PUB 1: 0.1846419553728691
>>> Expectation values for PUB 2: 0.09479649665121072
>>> Standard errors for PUB 2: 0.121401747916681


### Run parameterized circuits

Use Estimator to run three experiments in a single job, leveraging parameter values to increase circuit reusability.

In [3]:
import numpy as np

from qiskit.circuit import QuantumCircuit, Parameter
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator

service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)

# Step 1: Map classical inputs to a quantum problem
theta = Parameter("θ")

chsh_circuit = QuantumCircuit(2)
chsh_circuit.h(0)
chsh_circuit.cx(0, 1)
chsh_circuit.ry(theta, 0)

number_of_phases = 21
phases = np.linspace(0, 2 * np.pi, number_of_phases)
individual_phases = [[ph] for ph in phases]

ZZ = SparsePauliOp.from_list([("ZZ", 1)])
ZX = SparsePauliOp.from_list([("ZX", 1)])
XZ = SparsePauliOp.from_list([("XZ", 1)])
XX = SparsePauliOp.from_list([("XX", 1)])
ops = [ZZ, ZX, XZ, XX]

# Step 2: Optimize problem for quantum execution.

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
chsh_isa_circuit = pm.run(chsh_circuit)
isa_observables = [
    operator.apply_layout(chsh_isa_circuit.layout) for operator in ops
]

# Step 3: Execute using Qiskit primitives.

# Reshape observable array for broadcasting
reshaped_ops = np.fromiter(isa_observables, dtype=object)
reshaped_ops = reshaped_ops.reshape((4, 1))

estimator = Estimator(backend, options={"default_shots": int(1e4)})
job = estimator.run([(chsh_isa_circuit, reshaped_ops, individual_phases)])
# Get results for the first (and only) PUB
pub_result = job.result()[0]
print(f">>> Expectation values: {pub_result.data.evs}")
print(f">>> Standard errors: {pub_result.data.stds}")
print(f">>> Metadata: {pub_result.metadata}")

>>> Expectation values: [[ 1.00457852  0.94374224  0.79510563  0.57426771  0.29972451 -0.04434586
  -0.33693938 -0.61014552 -0.83254333 -0.97070631 -1.00948108 -0.95510726
  -0.80669349 -0.57203928 -0.31086669  0.04546007  0.3396135   0.61616229
   0.84190276  0.97159768  1.01170951]
 [ 0.0291925   0.32379161  0.63064712  0.84457688  0.95911845  1.00591558
   0.94975902  0.80491074  0.57582762  0.2812285   0.00178275 -0.33003123
  -0.60992267 -0.8370002  -0.96825503 -1.00747549 -0.95332451 -0.78374061
  -0.58518704 -0.27342898  0.02896966]
 [-0.01760464 -0.33916781 -0.63599536 -0.82630372 -0.98006573 -1.02040041
  -0.95533011 -0.82117832 -0.57605046 -0.27521173  0.02896966  0.33560232
   0.62574456  0.83945148  0.9814028   1.02485728  0.95466157  0.81337879
   0.5813987   0.28278841 -0.02941534]
 [ 1.00814402  0.97026062  0.80669349  0.56201132  0.30284432 -0.04189458
  -0.34161909 -0.62217907 -0.83321186 -0.97182052 -1.01995472 -0.96669512
  -0.8098133  -0.57159359 -0.28969656  0.0318

### Use sessions and advanced options

Explore sessions and advanced options to optimize circuit performance on QPUs.

<CloudContent>
<Admonition type="caution">
The following code block will return an error for Open Plan users because it uses sessions. Open Plan workloads can run only in [job mode](/docs/guides/execution-modes#job-mode) or [batch mode](/docs/guides/execution-modes#batch-mode).
</Admonition>
</CloudContent>

In [4]:
import numpy as np
from qiskit.circuit.library import iqp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp, random_hermitian
from qiskit_ibm_runtime import (
    QiskitRuntimeService,
    Session,
    EstimatorV2 as Estimator,
)

n_qubits = 50

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

rng = np.random.default_rng(1234)
mat = np.real(random_hermitian(n_qubits, seed=rng))
circuit = iqp(mat)
mat = np.real(random_hermitian(n_qubits, seed=rng))
another_circuit = iqp(mat)
observable = SparsePauliOp("X" * 50)
another_observable = SparsePauliOp("Y" * 50)

pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
isa_circuit = pm.run(circuit)
another_isa_circuit = pm.run(another_circuit)
isa_observable = observable.apply_layout(isa_circuit.layout)
another_isa_observable = another_observable.apply_layout(
    another_isa_circuit.layout
)

with Session(backend=backend) as session:
    estimator = Estimator(mode=session)

    estimator.options.resilience_level = 1

    job = estimator.run([(isa_circuit, isa_observable)])
    another_job = estimator.run(
        [(another_isa_circuit, another_isa_observable)]
    )
    result = job.result()
    another_result = another_job.result()

    # first job
    print(f" > Expectation value: {result[0].data.evs}")
    print(f" > Metadata: {result[0].metadata}")

    # second job
    print(f" > Another Expectation value: {another_result[0].data.evs}")
    print(f" > More Metadata: {another_result[0].metadata}")

 > Expectation value: -0.12609457092819615
 > Metadata: {'shots': 4096, 'target_precision': 0.015625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32}
 > Another Expectation value: -0.0215633423180593
 > More Metadata: {'shots': 4096, 'target_precision': 0.015625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32}


<span id="sampler-examples"></span>
## Sampler examples

Generate entire error-mitigated quasi-probability distributions sampled from quantum circuit outputs. Leverage Sampler’s capabilities for search and classification algorithms like Grover’s and QVSM.

### Run a single experiment

Use Sampler to return the measurement outcome as bitstrings or counts of a single circuit.

In [5]:
import numpy as np
from qiskit.circuit.library import iqp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.quantum_info import random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

n_qubits = 127

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

mat = np.real(random_hermitian(n_qubits, seed=1234))
circuit = iqp(mat)
circuit.measure_all()

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)

sampler = Sampler(backend)
job = sampler.run([isa_circuit])
result = job.result()

# Get results for the first (and only) PUB
pub_result = result[0]

print(f" > First ten results: {pub_result.data.meas.get_bitstrings()[:10]}")

 > First ten results: ['1001100101111110110010010100000000010110111000100100101011110111001110111110011000010101001010101010011001110010000110000011000', '0111101000111010100000011101100101100000000100111110001101001010000011001110110110001110010110111001000100000000001100000010000', '0110111101010100111001011001011101110110101100000101100100101110100100100100100010011000010011111001010010000010100010100000000', '1001011111110110101010000010001100001000001011100001011101101100000000001100100100011100100000101000111110011010000111100000000', '0011010000101001001100111001011001000100001000000100100100001001111100101111100010010010111000011000000100010011000000100010100', '1101110001000110011011110000110001001010001001011111001100100010110000010111001000011100001011011001010000000110001010001001000', '0111111011110000001101001100000000100110000110010111010101000001011010101001011011011101010000100101000001010100000110000000000', '10011011100010000000010101011011011101000000100100110010011

### Run multiple experiments in a single job

Use Sampler to return the measurement outcome as bitstrings or counts of multiple circuits in one job.

In [6]:
import numpy as np
from qiskit.circuit.library import iqp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.quantum_info import random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

n_qubits = 127

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

rng = np.random.default_rng()
mats = [np.real(random_hermitian(n_qubits, seed=rng)) for _ in range(3)]
circuits = [iqp(mat) for mat in mats]
for circuit in circuits:
    circuit.measure_all()

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuits = pm.run(circuits)

sampler = Sampler(mode=backend)
job = sampler.run(isa_circuits)
result = job.result()

for idx, pub_result in enumerate(result):
    print(
        f" > First ten results for pub {idx}: {pub_result.data.meas.get_bitstrings()[:10]}"
    )

 > First ten results for pub 0: ['0010110001001111100000011001101111011001001000110001010001111000010111000011101100000100101010011110001101000100001100010001001', '0010101111101110010101000011000011110000101011100011101001100010001111111101010010010100010001011001011101010100011001011001011', '0010110011100111101011001100100001110000101110101101100101110011011101011010000100010010100100001011001110110100111001010001101', '0111111000011100001000100011110011010101010001111000111011100110011000111001111101100101000010011111000000010111011101101000100', '0100001110010001001101001101011011000101001100110000100010101010011010101101001110001000010110010011010101010100010000011000000', '0100111001001100011100000100110100100101100010000010101001101001001010000010000100010010010100000000010100010011110010101000100', '0100011011111100100000001001010000101100110011011111010001000001001011001010101100001010110000011010011100000110100101000000001', '0001110011111111011000010111001101010011011000100

### Run parameterized circuits

Run several experiments in a single job, leveraging parameter values to increase circuit reusability.

In [7]:
import numpy as np
from qiskit.circuit.library import real_amplitudes
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

n_qubits = 127

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

# Step 1: Map classical inputs to a quantum problem
circuit = real_amplitudes(num_qubits=n_qubits, reps=2)
circuit.measure_all()

# Define three sets of parameters for the circuit
rng = np.random.default_rng(1234)
parameter_values = [
    rng.uniform(-np.pi, np.pi, size=circuit.num_parameters) for _ in range(3)
]

# Step 2: Optimize problem for quantum execution.

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)

# Step 3: Execute using Qiskit primitives.
sampler = Sampler(backend)
job = sampler.run([(isa_circuit, parameter_values)])
result = job.result()
# Get results for the first (and only) PUB
pub_result = result[0]
# Get counts from the classical register "meas".
print(
    f" >> First ten results for the meas output register: {pub_result.data.meas.get_bitstrings()[:10]}"
)

 >> First ten results for the meas output register: ['0100100000010000100010000111111011101111110111010001010011001001000000101101100000100111010100110001001110010010011010011100001', '1101011011011011100001100000111101000011110010111101111110111001011110101111000100001010000001010000011111100111011001010000111', '1100010011101111010001011110110010111100001110010011001110011110000110111110101000001110001111101001111101010011000011111110100', '0001111010010100101101100100110111010101101010010001100111011110001111010101101110010001110101011100110100001111011100110000110', '1000010000011011110001000101100111101001100011001001011100101110111000101010101000100000010111101101100111001010011001010001111', '1101110001011111000010001011111011000011100101000011010110011001000100110011101010101100000001101111101010001010110110011010011', '0111010000010110000110100100110111100101111000000001111011111001100011001010101000111010110001101001110011111101110100000100001', '00000100010111111001010101011

### Use sessions and advanced options

Explore sessions and advanced options to optimize circuit performance on QPUs.

<CloudContent>
<Admonition type="caution">
The following code block will return an error for users on the Open Plan, because it uses sessions. Workloads on the Open Plan can run only in [job mode](/docs/guides/execution-modes#job-mode) or [batch mode](/docs/guides/execution-modes#batch-mode).
</Admonition>
</CloudContent>

In [8]:
import numpy as np
from qiskit.circuit.library import iqp
from qiskit.quantum_info import random_hermitian
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_ibm_runtime import Session, SamplerV2 as Sampler
from qiskit_ibm_runtime import QiskitRuntimeService

n_qubits = 127

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

rng = np.random.default_rng(1234)
mat = np.real(random_hermitian(n_qubits, seed=rng))
circuit = iqp(mat)
circuit.measure_all()
mat = np.real(random_hermitian(n_qubits, seed=rng))
another_circuit = iqp(mat)
another_circuit.measure_all()

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)
another_isa_circuit = pm.run(another_circuit)

with Session(backend=backend) as session:
    sampler = Sampler(mode=session)
    job = sampler.run([isa_circuit])
    another_job = sampler.run([another_isa_circuit])
    result = job.result()
    another_result = another_job.result()

# first job

print(
    f" > The first ten measurement results of job 1: {result[0].data.meas.get_bitstrings()[:10]}"
)

 > The first ten measurement results of job 1: ['1000110010100111000100001000110001101010010100100101010101011000010101110000010010100001101000100000101111001100001000100010010', '0001101000101001001010100001011000000000111101110001100111110110111111000101010000100100000000000101100111101110011011100000000', '1000011011001111010011011100110011000100001101101000000101001100011100100110010010100000101101010010100011000010010000001000000', '0001011011110000101001100100100010000000111101001001010100001100000000010000000111100010010011100000100100100000011010010000000', '0101111111111110001010110000000001111000000100110000000100000100000100100010110110010100001010100000001100100011011111101000100', '0110111110001000000101001001110001010000011000010011000111000001100000001111000010001000111010101001000101111110100000000000110', '0111110010011110010001111010010010000010101011010000010101001100101101110000010110100000100000100000100001100101000010110000000', '0111111111010011001000100110000110

In [9]:
# second job
print(
    " > The first ten measurement results of job 2:",
    another_result[0].data.meas.get_bitstrings()[:10],
)

 > The first ten measurement results of job 2: ['0001111001010000111111000001001100011111100100000010111010010000100001010011010101010100010001110010100000010101100011011100000', '1010000011010010011000001100110000011000010000001001010010110110100100111001101000111000000011111010011010001000101101000010000', '1100101010001111001100101100000100011011111010101000101010010110111001011110100001001011111101110000010000100110001100011011000', '0110010110011101111100001000110010010011110100101010100000011000101000100010011001000110100100101000011010010000100000101101000', '0000000100000000100000000110000001000101111100101101010111001100110111110110001000110010000011000001000010100110100010100000100', '1111001000001011110100001000000110010000010000111001010000000000100100101101100001000110111000010100000101100000011000011010110', '1001001000011111010101011010000011110011001110100101001111100100100101001001001100000001110010110001000010000000001011001001000', '1111110010001100101111001110010100

## Next steps

<Admonition type="tip" title="Recommendations">
    - [Specify advanced runtime options.](runtime-options-overview)
    - Practice with primitives by working through the [Cost function lesson](/learning/courses/variational-algorithm-design/cost-functions) in IBM Quantum Learning.
    - Learn how to transpile locally in the [Transpile](./transpile/) section.
    - Try the [Compare transpiler settings](/docs/tutorials/circuit-transpilation-settings) tutorial.
    - Read [Migrate to V2 primitives](/docs/migration-guides/v2-primitives).
    - Understand the [Job limits](/docs/guides/job-limits) when sending a job to an IBM&reg; QPU.
</Admonition>